# Cálculo Numérico - 2021.2 - Tarefa 7 - Professor João Paixão
## Realizado por: David Rodrigues Albuquerque - 120047390
## Realizado com: Beatriz Almeida Ramos, Matheus Barroso

In [ ]:
using Plots
using LinearAlgebra
using Random
using Polynomials

## 2) Achar valor aproximado da função usando derivadas

### i)

In [ ]:
# Entrada um valor qualquer x, um valor qualquer a, um vetor contendo as informações das derivadas em a, maior valor da n-derivada no intervalo 
# Saida o valor aproximado da função no ponto x e o erro máximo
# Utiliza o teorema de Taylor de ordem zero para nos dar um valor aproximado de f(x)
function Taylor0(x,a, derivadas, M)#x é o valor que será avaliado
    E = M*(x-a)
    f_a=derivadas[1] # como na primeira casa do vetor tem o valor da função no ponto a, podemos fazer essa atibuição.
    return f_a, E
end

In [ ]:
# Exemplo 1
#e^x x=0.1 a=0
derivadas = [1]
result, E =Taylor0(0.1, 0, derivadas, 10)
print(abs(result-exp(0.1))< E)

In [ ]:
# Exemplo 2
#log(x) x=1.01 a=1
derivadas = [0]
result, E =Taylor0(1.01, 1, derivadas, 2)
print(abs(result-log(1.01))<E)


In [ ]:
# Exemplo 3

function ex(x)
return 7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1
end    

#7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1, x=2.1666666666, a=2
derivadas = [757]
result, E = Taylor0(2.166666666, 2, derivadas, 2630)
print(abs(result-ex(2.166666666))<E)


### ii)

In [ ]:
# Entrada um valor qualquer x, um valor qualquer a, um vetor contendo as informações das derivadas em a, maior valor da segunda derivada no intervalo 
# Saida o valor aproximado da função no ponto x e o erro máximo
# Utiliza o teorema de Taylor de ordem um para nos dar um valor aproximado de f(x)
function Taylor1(x,a, derivadas, M)#x é o valor que será avaliado
    E = (M*(x-a)^2)/2
    f_a=derivadas[1] + derivadas[2]*(x-a)
    return f_a, E
end

In [ ]:
# Exemplo 1
#e^x x=0.1 a=0
derivadas = [1, 1]
result, E =Taylor1(0.1, 0, derivadas, 10)
print(abs(result-exp(0.1))< E)

In [ ]:
# Exemplo 2
#log(x) x=1.01 a=1
derivadas = [0, 1]
result, E =Taylor1(1.01, 1, derivadas, 2)
print(abs(result-log(1.01))<E)


In [ ]:
# Exemplo 3

function ex(x)
    return 7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1
    end    
    
function ex1(x)
    return 42x^5 + 40*x^4 + 16*x^3 + 3*x^2 -12*x +2
end   

function ex2(x)
    return 210x^4 + 160*x^3 + 48*x^2 + 6*x -12
end       

    #7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1, x=2.1666666666, a=2
    derivadas = [757, 2102]
    result, E = Taylor1(2.166666666, 2, derivadas, 6856)
    
    print(abs(result-ex(2.166666666))<E)
   

### iii)

In [ ]:
# Entrada um valor qualquer x, um valor qualquer a, um vetor contendo as informações das derivadas em a, maior valor da terceira derivada no intervalo 
# Saida o valor aproximado da função no ponto x e o erro máximo
# Utiliza o teorema de Taylor de ordem dois para nos dar um valor aproximado de f(x)
function Taylor2(x,a, derivadas, M)#x é o valor que será avaliado
    E = (M*(x-a)^3)/6
    f_a=derivadas[1] + derivadas[2]*(x-a) +(derivadas[3]*(x-a)^2)/2
    return f_a, E
end

In [ ]:
# Exemplo 1
#e^x x=0.1 a=0
derivadas = [1, 1, 1]
result, E =Taylor2(0.1, 0, derivadas, 2)
print(abs(result-exp(0.1))< E)

In [ ]:
# Exemplo 2
#log(x) x=1.01 a=1
derivadas = [0, 1, -1]
result, E =Taylor2(1.01, 1, derivadas, 3)
print(abs(result-log(1.01))<E)


In [ ]:
# Exemplo 3

function ex(x)
    return 7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1
    end    
    
function ex1(x)
    return 42x^5 + 40*x^4 + 16*x^3 + 3*x^2 -12*x +2
end   

function ex2(x)
    return 210x^4 + 160*x^3 + 48*x^2 + 6*x -12
end      

function ex3(x)
    return 840x^3 + 480*x^2 + 96*x + 6
end   

    #7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1, x=2.1666666666, a=2
    derivadas = [757, 2102, 4832]
    result, E = Taylor2(2.166666666, 2, derivadas, 11484)
    
    print(abs(result-ex(2.166666666))<E)
   

### iv)

In [ ]:
# Entrada um valor qualquer x, um valor qualquer a, um vetor contendo as informações das derivadas em a, maior valor da n-derivada no intervalo 
# Saida o valor aproximado da função no ponto x e o erro máximo
# Utiliza o teorema de Taylor de ordem n para nos dar um valor aproximado de f(x)
function TaylorN(x,a, derivadas, M)#x é o valor que será avaliado
    n, = size(derivadas)
    #ordem n-1
    E = (M*(x-a)^n)/(factorial(n-1))
    soma=0
    for i in 1:n
        soma+= (derivadas[i]*(x-a)^(i-1))/factorial(i-1)
    end
    return soma, E
end

In [ ]:
# Exemplo 1
#e^x x=0.1 a=0
derivadas = [1, 1, 1, 1, 1, 1, 1, 1]
result, E =TaylorN(0.1, 0, derivadas, 2)
print(abs(result-exp(0.1))< E)

In [ ]:
# Exemplo 2
#log(x) x=1.01 a=1
derivadas = [0, 1, -1, 2, -6]
result, E =TaylorN(1.01, 1, derivadas, 7)
print(abs(result-log(1.01))<E)


In [ ]:
# Exemplo 3

function ex(x)
    return 7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1
end    
    
function ex1(x)
    return 42*x^5 + 40*x^4 + 16*x^3 + 3*x^2 -12*x +2
end   

function ex2(x)
    return 210*x^4 + 160*x^3 + 48*x^2 + 6*x -12
end      

function ex3(x)
    return 840*x^3 + 480*x^2 + 96*x + 6
end   

function ex4(x)
    return 2520*x^2 + 960*x + 96
end  

function ex5(x)
    return 5040*x + 960
end  

    #7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1, x=2.1666666666, a=2
    derivadas = [757, 2102, 4832, 8838, 12096, 11040]
    result, E = TaylorN(2.166666666, 2, derivadas, 5040)
    
    print(abs(result-ex(2.166666666))<E)

## 3) Encontrar raízes aproximadas

### i)

In [ ]:

function média(x,y)#retorna a mádia entre x e y
    return (x+y)/2
end

function tem_sinais_opostos(f,a,b) #função recebe f(a) e f(b) não nulos e retorna true se f(a) e f(b) possuem sinais opostos, e retorna false caso contrário
    return f(a)*f(b) < 0
end

function é_raiz(f,a)#função que recebe uma função e um valor a, e verifica se o valor a é o zero da função
    return f(a)==0
end


#função que recebe uma função f, um intervalo [a, b] e u erro máximo 
#a função retorna um valor aproximado do zero da finção com a condição 
#de que a diferença entre a aproximação e o valor real seja menor do que o erro
function bissecao(f, a, b, erro) 
   
        
    if é_raiz(f,a)
        return a
    end
    if é_raiz(f,b)
        return b
    end
    if !(tem_sinais_opostos(f,a,b))
        return "Não tem sinais opostos"
    end    
    iterações = floor(log2((b-a)/erro))+1
    for i=1:iterações
        m=média(a,b)
        if é_raiz(f,m)
            return m
        end
            
        if tem_sinais_opostos(f,a,m)
            b=m
        else
            a=m
        end
    end
        
    return média(a,b)
end

 

### ii)

In [ ]:
# Entrada função, pontos a e, erro esperado
# Saida true  se a matriz A eh quadrada e se o numero de linhas da matriz B eh o mesmo numero de colunas da matriz A. False, caso contrario
# Verifica se a matriz A eh quadrada e se o numero de linhas da matriz B eh o mesmo numero de colunas da matriz A
function newton(funcao, a, b, erro)#funcao que implementa o método de newton
    
    chute_inicial=ceil(bissecao(funcao, a, b, erro))
    
    t = derivar_centralizada1(chute_inicial, funcao, 0.01)
    
    a= funcao(chute_inicial)
    
    return chute_inicial - a/t(chute_inicial)
end

function g(x)
    return x^2 - 4
end

newton(g, -0, 4, 0.1)

## 4) Interpolação Polinomial


### i) Interpolação Com Vandermonte

In [ ]:
# Entrada: n pontos x e y
# Saída: Um polinômio F(x), com grau no máximo n, tal que para todo 1<=i<=n, F(x_i)=y_i
# A partir de x e y serã nos retornado uma nova função polinomial onde F(x_i)=y_i
function interpolação(x,y)# função que calcula os coeficientes do polinomio interpolador, a partir da entrada de x e y
    n, = size(x)
    V=zeros(n, n)
    for i in 1:n
        for j in 1:n
            V[i,j]=x[i]^(j-1)
        end
    end
    c=inv(V)*y #resolver o sistema linear Vc=y
    #vetor de coeficientes
    function fp(t)
        soma = 0
        for i in 1:n
            soma+=c[i]*t^(i-1)
        end
        return soma    
    end
    return fp
end



In [ ]:
# Exemplo 1
x = [15.5; 16.5; 17.5]
y = [34; 200; 25]
fp=interpolação(x,y)
scatter(x, y, c=:lightblue, ms=3, leg=false)
plot!(fp)

In [ ]:
# Exemplo 2
x = rand(5)
y = rand(5)
fp=interpolação(x,y)
scatter(x, y, c=:lightblue, ms=3, leg=false)
plot!(fp)


In [ ]:
function ex(x)
    return 7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1
end 
x = rand(7)*10
y = ex.(x)
fp=interpolação(x,y)
scatter(x, y, c=:lightblue, ms=3, leg=false)
plot!(fp)
plot!(ex)


### ii) Interpolação com Lagrange

In [ ]:
#Entrada: Um conjunto de pares x e y
#Saida: uma função que representa o polinômio interpolador, calculado utilizando o polinômio de Lagrange
function interpolaçãoLagrange(x,y)# função que calcula os coeficientes do polinomio interpolador, a partir da entrada de x e y
    n, = size(x)
    L =zeros(n, 1)
    
    function f_ip(t)
        expressao=0
        for i in 1:n
            numerador=1
            denominador=1
            for j in 1:n
                if(i!=j)
                    numerador*=(t-x[j])
                    denominador*=(x[i] - x[j])
                end  
            end
            L[i]=numerador/denominador
            expressao+=L[i]*y[i]
            
        end
        return expressao
    end
    
    return f_ip
end


In [ ]:
# Exemplo 1
x = [15.5; 16.5; 17.5]
y = [34; 200; 25]
fp=interpolaçãoLagrange(x,y)
scatter(x, y, c=:lightblue, ms=3, leg=false)
plot!(fp)

In [ ]:
# Exemplo 2
x = rand(5)
y = rand(5)
fp=interpolaçãoLagrange(x,y)
scatter(x, y, c=:lightblue, ms=3, leg=false)
plot!(fp)

In [ ]:
# Exemplo 3
function ex(x)
    return 7x^6 + 8*x^5 + 4*x^4 + x^3 -6x^2 +2*x + 1
end 
x = rand(7)*10
y = ex.(x)
fp=interpolaçãoLagrange(x,y)
scatter(x, y, c=:lightblue, ms=3, leg=false)
plot!(fp)
plot!(ex)

## 5) Regressão polinomial

In [ ]:
# Entrada: um conjunto de pontos (x,y) quaisquer e o grau da regressão linear 
# Saida: um função polinomial do grau de entrada com a melhor aproximação para os pontos (x,y) dados
# Utiliza a regressão na forma polinomial, resolvendo o sistema pelo método de mínimos quadrados
function regressão(x,y,grau)
    V=vandermonde(x,y,grau)
    c=inv(V'*V)*(V'*y) #mínimos quadrados
    #c=V\y
    return c, grau
end

function vandermonde(x,y,grau)
    n,=size(y)
    V=zeros(n,grau+1)
    for i=1:n #linhas
        for j=1:(grau+1)
            V[i,j]=x[i]^(j-1)
        end
    end
    return V
end

function plota_regressão(regressao, grau)
    coefs = Float64[]
    p = Any[]
    for i=1:grau+1 #linhas
        push!(coefs, regressao[i])
        p = Polynomial(coefs)
    end
    return p
end

In [ ]:
# Exemplo 1
Random.seed!(0)
# Gerando dados fictícios para teste
n = 30
x = range(0, 2, length=n)

f(x) = (x-1)^3

y = f.(x) + randn(n)*0.05 #ruído
scatter(x, y, c=:lightblue, ms=3, leg=false)
c, grau = regressão(x,y,3)
p3 = plota_regressão(c, 3)
plot!(p3, 0, 2)

In [ ]:
# Exemplo 2
Random.seed!(0)
# Gerando dados fictícios para teste
n = 30
x = range(0, 2, length=n)

f(x) = (x-1)^4

y = f.(x) + randn(n)*0.05 #ruído
scatter(x, y, c=:lightblue, ms=3, leg=false)
c, grau = regressão(x,y,4)
p4 = plota_regressão(c, 4)
plot!(p4, 0, 2)

In [ ]:
# Exemplo 3
Random.seed!(0)
# Gerando dados fictícios para teste
n = 30
x = range(0, 2, length=n)

f(x) = (x-1)^5

y = f.(x) + randn(n)*0.05 #ruído
scatter(x, y, c=:lightblue, ms=3, leg=false)
c, grau = regressão(x,y,5)
p5 = plota_regressão(c, 5)
plot!(p5, 0, 2)

## 6) Regressão com coeficientes lineares

In [ ]:
# Entrada: um conjunto de pontos (x,y) quaisquer e um conjunto de funções
# Saida: um função gerada utilizando as funções como coeficientes com a melhor aproximação para os pontos (x,y) dados
# Utiliza a regressão na forma geral, resolvendo o sistema pelo método de mínimos quadrados
function regressão_geral(x,y,funcoes)
    V=vandermonde_geral(x,y,funcoes)
    c=inv(V'*V)*(V'*y) #mínimos quadrados
    #c=V\y
    return c
end

function vandermonde_geral(x,y,funcoes)
    n,=size(y)
    num_funcoes = size(funcoes)
    V=zeros(n,num_funcoes[1])
    for i=1:n #linhas
        for j=1:(num_funcoes[1])
            V[i,j]=funcoes[j](x[i])
        end
    end
    return V
end

function plota_regressão_generalizada(regressao, funcoes)
    num_funcoes = size(funcoes)
    function fp(t)
        p = 0
        for i=1:num_funcoes[1]
            p = p + regressao[i]*funcoes[i](t)
        end
        return p
    end
    return fp
end

In [ ]:
# Exemplo 1
# Gerando dados fictícios para teste
Random.seed!(0)
n = 30
x = range(0, 2, length=n)

f(x) = (x-1)^3

y = f.(x) + randn(n)*0.05 #ruído
scatter(x, y, c=:lightblue, ms=3, leg=false)
f1(x) = sin(2x)
f2(x) = cos(4x)
f3(x) = exp(3x)
f4(x) = sin(x)
funcoes = [f1;f2;f3;f4]
c = regressão_geral(x,y,funcoes)
p3 = plota_regressão_generalizada(c, funcoes)
plot!(p3, 0, 2)

In [ ]:
# Exemplo 2
# Gerando dados fictícios para teste
Random.seed!(0)
n = 30
x = range(0, 2, length=n)

f(x) = (x-1)^4

y = f.(x) + randn(n)*0.05 #ruído
scatter(x, y, c=:lightblue, ms=3, leg=false)
f1(x) = sin(2x)
f2(x) = cos(4x)
f3(x) = exp(3x)
f4(x) = sin(x)
funcoes = [f1;f2;f3;f4]
c= regressão_geral(x,y,funcoes)
p4 = plota_regressão_generalizada(c,funcoes)
plot!(p4, 0, 2)

In [ ]:
# Exemplo 3
# Gerando dados fictícios para teste
Random.seed!(0)
n = 30
x = range(0, 2, length=n)

f(x) = (x-1)^8

y = f.(x) + randn(n)*0.05 #ruído
scatter(x, y, c=:lightblue, ms=3, leg=false)
f1(x) = sin(2x)
f2(x) = cos(4x)
f3(x) = exp(3x)
f4(x) = sin(x)
funcoes = [f1;f2;f3;f4]
c = regressão_geral(x,y,funcoes)
p4 = plota_regressão_generalizada(c, funcoes)
plot!(p4, 0, 2)

## 7) Regressão com coeficientes não-lineares

### Modelo exponencial

Queremos encontrar $y$ tal que $y = c1*e^{c2x}$ 

Fazendo ln dos dois lados:

$$ln(y) = ln(c1 \times e^{c2x})$$
$$ ln(y) = ln(c1) + ln(e^{c2x})$$
$$ln(y) = ln(c1) + c2xln(e)$$
$$ln(y) = ln(c1) + c2x$$

Dicionário:

$y\_barra = ln(y);  c1\_barra = ln(c1);  x\_barra = x;  c2\_barra = c2$

In [ ]:
# Exemplo 1
#y=2^x
x=[1;2;3;4;5;6;7;8;9;10]
y=[2;4;8;16;32;64;128;256;512+40;1024]
ruido=randn(10)*0.05
y=y+ruido

# Dicionário (troca de variável) indo pro mundo barra
x_barra=x
y_barra=log.(y)
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)
c1=exp(c_barra[1])
c2=c_barra[2]
exponencial(x)=c1*exp(c2*x)

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(exponencial) # com regressão

In [ ]:
# Exemplo 2
#y=3^x
x = rand(10)
f(x) = 3^x
y = f.(x) + randn(10)*0.05 #ruído

# Dicionário (troca de variável) indo pro mundo barra
x_barra=x
y_barra=log.(y)
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)
c1=exp(c_barra[1])
c2=c_barra[2]
exponencial(x)=c1*exp(c2*x)

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(exponencial) # com regressão

In [ ]:
# Exemplo 3
#y=4^x
x = rand(10)
f(x) = 4^x
y = f.(x) + randn(10)*0.05 #ruído

# Dicionário (troca de variável) indo pro mundo barra
x_barra=x
y_barra=log.(y)
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)
c1=exp(c_barra[1])
c2=c_barra[2]
exponencial(x)=c1*exp(c2*x)

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(exponencial) # com regressão

### Modelo potência

Queremos encontrar $y$ tal que $y = c0x^{c1}$

Fazendo ln dos dois lados:

$$ln(y) = ln(c0x^{c1})$$
$$ln(y) = ln(c0) + ln(x^{c1})$$
$$ln(y) = ln(c0) + c1ln(x)$$ 

Dicionário:

$y\_barra = ln(y);  c0\_barra = ln(c0);  x\_barra = ln(x);  c1\_barra = c1$

In [ ]:
# Exemplo 1
x = [0.1; 0.2; 0.4; 0.8; 0.9]
y = [22; 43; 84; 210; 320]
# indo pro mundo barra, a partir do dicionário descrito acima
y_barra = log.(y)
x_barra = log.(x)

# Regressão no Mundo barra
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)

# Dicionário (troca de variável) voltando pro mundo original, 
# lembrando que 𝑐0 barra = ln(c0); 𝑐1 barra= c1;
c0=exp(c_barra[1])
c1=c_barra[2]
f(x) = c0*x^c1 

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(f) # com regressão

In [ ]:
# Exemplo 2
x = rand(10)
f(x) = 2^x
y = f.(x) + randn(10)*0.05 #ruído
# indo pro mundo barra, a partir do dicionário descrito acima
y_barra = log.(y)
x_barra = log.(x)

# Regressão no Mundo barra
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)

# Dicionário (troca de variável) voltando pro mundo original, 
# lembrando que 𝑐0 barra = ln(c0); 𝑐1 barra= c1;
c0=exp(c_barra[1])
c1=c_barra[2]
f(x) = c0*x^c1 

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(f) # com regressão

In [ ]:
# Exemplo 3
x = rand(10)
f(x) = 3^x
y = f.(x) + randn(10)*0.05 #ruído
# indo pro mundo barra, a partir do dicionário descrito acima
y_barra = log.(y)
x_barra = log.(x)

# Regressão no Mundo barra
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)

# Dicionário (troca de variável) voltando pro mundo original, 
# lembrando que 𝑐0 barra = ln(c0); 𝑐1 barra= c1;
c0=exp(c_barra[1])
c1=c_barra[2]
f(x) = c0*x^c1 

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(f) # com regressão

### Modelo geométrico

Queremos encontrar $y$ tal que $y = \dfrac{1}{c0+c1x}$

Elevando ambos os lados a $-1$:

$$y = \dfrac{1}{c0+c1x}$$
$$\dfrac{1}{y} = \dfrac{1}{c0+c1x}$$

Dicionário:

$y\_barra = \dfrac{1}{y};  c0\_barra = c0;  x\_barra = \dfrac{1}{x};  c1\_barra = c1$

In [ ]:
# Exemplo 1
#y=1/2x
x=[1;2;3;4;5;6;7;8;9;10]
y=[1/2;1/4;1/6;1/8;1/10;1/12;1/14;1/16;1/18;1/20]
ruido=randn(10)*0.02
y=y+ruido

# Dicionário (troca de variável) indo pro mundo barra
x_barra=(x)
y_barra=inv.(y)
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)
c1=(c_barra[1])
c2=(c_barra[2])
geometrica(x)=(1/(c1+(c2*x)))

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(geometrica) # com regressão

In [ ]:
# Exemplo 2
#y=3/-x
x=[1;2;3;4;5;6;7;8;9;10]
y=[-3;-3/2;-3/3;-3/4;-3/5;-3/6;-3/7;-3/8;-3/9;-3/10]
ruido=randn(10)*0.1
y=y+ruido

# Dicionário (troca de variável) indo pro mundo barra
x_barra=(x)
y_barra=inv.(y)
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)
c1=(c_barra[1])
c2=(c_barra[2])
geometrica(x)=(1/(c1+(c2*x)))

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(geometrica) # com regressão

In [ ]:
# Exemplo 3
#y=4/-3x
x=[1;2;3;4;5;6;7;8;9;10]
y=[-4/3;-4/6;-4/9;-4/12;-4/15;-4/18;-4/21;-4/24;-4/27;-4/30]
ruido=randn(10)*0.1
y=y+ruido

# Dicionário (troca de variável) indo pro mundo barra
x_barra=(x)
y_barra=inv.(y)
V=vandermonde(x_barra,y_barra,1)
c_barra, grau=regressão(x_barra,y_barra,1)
c1=(c_barra[1])
c2=(c_barra[2])
geometrica(x)=(1/(c1+(c2*x)))

scatter(x, y, c=:lightblue, ms=3, leg=false) #original
plot!(geometrica) # com regressão

## 8) Interpolação 2D

In [ ]:
x=[15; 16.5; 17.5; 8]
y =[34; 30; 25; 16]
function interpola(x,y,z)# função que calcula os coeficientes do polinomio interpolador, a partir da entrada de x e y
    nx, = size(x)
    ny, = size(y)
    L =zeros(nx*ny, 1)
    function f_interpolacao_2d(t, w)
        expressao=0
        for i in 1:n
            numerador=1
            denominador=1
            
            for j in 1:n
                if(i!=j)
                    numerador*=(t-x[j])
                    denominador*=(x[i] - x[j])
                end  
            end

            L[i]=numerador/denominador
            expressao+=L[i]*y[i]
            
        end
        return expressao
    end
    
    return f_interpolacao_2d
end


x=[15.5; 16.5; 17.5]
y =[34; 200; 25]
f_interpolacao_2d=interpola(x,y)
t =range(0, stop=10)
plot(t,f)

## 9) Cálculo da norma do vetor


In [ ]:
# Entrada: um vetor v
# Saída: um número z tal que z = sqrt(v_1^2 +v_2^2 + … + v_n^2)
# A partir do vetor v nos é retornado sua norma
function norma(v)
    n, = size(v)
    soma = 0
    for i in 1:n
        soma+=v[i]^2

    end    
    return sqrt(soma)
end

In [ ]:
# Exemplo 1
x = rand(8)
norma(x) - norm(x) == 0

In [ ]:
# Exemplo 2
v = [154, 238, 459, -100]
norma(v) - norm(v) == 0

In [ ]:
# Exemplo 3
v = [-14, -143, 1442, 11, -199]
norma(v)  - norm(v) == 0

## 10) Resolver sistema linear denso

In [ ]:
function sistema_Aprox(A,b)
    A=A'*A
    b=A'*b
return inv(A)*b    
end

## 11) Resolver sistema linear denso

In [ ]:
function sistema_denso(A,b)
    #verificação de erro no tamanho das matrizes
    if (!verifica_tamanho(A,b))
        return "ERRO"
    end

    # chamamos a decomposicao de A em L e U
    L,U=lu_decomposition(A)
    
    y=sistema_triangular_inferior(L,b)
    x=sistema_triangular_superior(U,y)
    return x
end

## 12) Sistema triangular superior

In [ ]:
# Entrada matriz A e b
# Saida true  se a matriz A eh quadrada e se o numero de linhas da matriz B eh o mesmo numero de colunas da matriz A. False, caso contrario
# Verifica se a matriz A eh quadrada e se o numero de linhas da matriz B eh o mesmo numero de colunas da matriz A.
function verifica_tamanho(A,b)
    if (size(A)[1] != size(A)[2]) # Verificando se a matriz eh quadrada
        println("ERRO! Matriz A nao eh quadrada")
        return false
    end

    if (size(b)[1] != size(A)[2]) # Verificando se o numero de linhas de B eh o mesmo que de colunas de A
        println("ERRO! O numero de linhas da matriz B não eh o mesmo do numero de colunas da matriz A")
        return false
    end
    
    return true
end

In [ ]:
# Entrada matriz A e b
# Saída x tal que A * x = b
# Onde A é triangular superior aonde os elementos da diagonal são diferentes de zero e a parte inferior, por sua vez, eh composta por zeros
function sistema_triangular_superior(A,b) 
    # inicializacao de variaveis
    n,=size(A) 
    x = zeros(n, 1)
    #verificação de erro no tamanho das matrizes
    if (!verifica_tamanho(A,b))
      return "ERRO"
    end
  
    for i in reverse(1:n) # comecamos da ultima linha da matriz, o percorrendo "de baixo para cima"
      #na pratica isolamos a incognita da linha i, para descobrir o seu valor
      #como comecamos na ultima linha, a variavel dessa linha ja tem seu valor conhecido
      #a partir da penultima linha, as variaveis tem seus valores baseados na linha abaixo que áforam calculados. 
      soma = 0
      for j in i+1:n
          soma+=A[i, j]*x[j]
      end 
      x[i] = (b[i] - soma)/(A[i,i])
    end
  return x
  end
  

## 13) Sistema triangular inferior

In [ ]:
# Entrada matriz A e b
# Saida x tal que A * x = b
# onde A eh triangular inferior aonde os elementos da diagonal sao diferentes de zero e a parte superior, por sua vez, eh composta de zeros
function sistema_triangular_inferior(A,b) #
    # inicializacao de variaveis
    n,=size(A)
    x = zeros(n, 1)
  
    if (!verifica_tamanho(A,b))
      return "ERRO"
    end
  
    for i in 1:n
      soma = 0
  
      # comecamos da ultima linha da matriz, a percorrendo "de baixo para cima"
      # na pratica isolamos a incognita da linha i, para descobrir o seu valor
      # como comecamos na ultima linha, a variavel dessa linha ja tem seu valor conhecido
      # a partir da penultima linha, as variaveis tem seus valores baseados na linha abaixo que já foram calculados.     
      for j in 1:i-1
          soma+=A[i, j]*x[j]
      end 
      x[i] = (b[i] - soma)/(A[i,i])
    end
  return x
  end

## 14) Sistema diagonal

In [ ]:
# Entrada matriz A e b
# Saída x tal que A * x = b
# Onde A é a matriz em que a diagonal principal tem elementos diferentes de 0 e o restante dos elementos
# possuem valor 0, a funcao a seguir resolver esse caso.
function sistema_diagonal(A,b)
    # inicializacao de variaveis  
    n,=size(A)
    x = zeros(n, 1)
  
    if (!verifica_tamanho(A,b))
      return "ERRO"
    end
    
    for i in 1:n
      x[i] = b[i]/A[i,i] # Expressao que calcula  A * x = b
    end
  return x
  end

## 15) Inversa da matriz

In [ ]:
function matriz_inversa(A)
    n,=size(A)
    b = zeros(n, n)
    inverse = zeros(n,n)

    #verificação de erro no tamanho das matrizes
    if (!verifica_tamanho(A,b))
        return "ERRO"
    end

    for i in 1:n
        b[i,i] = 1
    end

    # chamamos a decomposicao de A em L e U
    L,U=lu_decomposition(A)
    for i in 1:n
        y=sistema_triangular_inferior(L,b[i,:])
        x=sistema_triangular_superior(U,y)
        inverse[[i],:]=x
    end
    
    return inverse'
end

## 16) Decomposição LU

In [ ]:
# Entrada: matriz A
# Saida: L, matriz triangular inferior de A e U matriz triangular superior de A
# Fazemos a decomposicao da matriz A em duas matrizes L e U, tal que A = L * U
function lu_decomposition(A)
    n,=size(A)
    U = copy(A)
    L = zeros(n, n)

    for i in 1:n-1 # for por coluna
        pivot = U[i,i]
        L[i,i] = 1
        for j in i+1:n # for da linha
            fator = U[j,i]/pivot;
            L[j,i] = fator
            for k in 1:n # for pra calcular o valor da linha
                U[j, k] -= fator * U[i,k] # calcula linha
            end
        end
    end

    L[n,n] = 1
    
    return L,U
end

            

## 17) Problema do valor do contorno

In [ ]:
function criação_da_matriz(n)
    A=zeros(n,n)
    
    #"manual"
    A[1,1]=-2
    A[1,2]=1
    A[n,n-1]=1
    A[n,n]=-2
    
    #tridiagonal
    for i= 2:(n-1)
        A[i,i]=-2
        A[i,i+1]=1
        A[i,i-1]=1
    end
    return A
end

In [ ]:
# função para calcular a segunda derivda de forma que receba os coeficientes e seja igual a c_1+c_2y(x)+c_3
function segunda_derivada(n, c_1, c_2, c_3, h, a)
    sd=zeros(n,1) 
    x=zeros(n,1)
    for i=1:(n)
        x[i]=a+h*(i-1) # calculo de um x bom
    end
    # for a seguir preenchendo a segunda derivada dos ys do meio, ou seja. só vai até n-2, 
    # pois exclui o primeiro e ultimo y
    for i=1:(n-2)
        sd[i]=c_1 + c_2*4*x[i+1] # o indice de x é i+1 pois os 
        # cálculos sao feitos com as derivadas do y2 até yn-1, sendo 2 até n-1 os x2 até xn-2.
        # como i vai de 1 até n-2, precisamos ir com o índice até i+1
    end
end

In [ ]:
function diferenças_finitas(n,c_1,c_2,c_3,y_inicial,y_final,x_inicial,x_final)
    h=(x_final-x_inicial)/(n-1)
    
    A=criação_da_matriz(n-2)
    sd=segunda_derivada()
    
    b=zeros(n-2)
    b[1]=sd[1]*2*h^2-y_inicial
    
    b[n-2]=sd[n-2]*2*h^2-y_final

    for i=2:(n-3) #no meio
        b[i]=sd[i]*2*h^2
    end
    
    y_meio=A\b
    return y_meio  #valores da função no "meio"
end

In [ ]:
n=7
y_a=5
y_b=20
a=0
b=10

x_bom=x
y_meio=diferenças_finitas(n,sd,y_inicial,y_final,x_inicial, x_final)
y_bom=[y_inicial;y_meio;y_final]

## 18) Integração numérica com número de intervalos

In [2]:
#Entrada: uma função f, numero de intervalos, e um intervalo [a,b]
#A função irá utilizar o método do trapézio para calcular a integral da função enviada por parametro
#no intervalo [a, b]
#Saida: aproximação integral de f no intervalo a[a, b]
function trapezio(f,n, a,b) #calcular a integral f(x) de a até b
    h=(b-a)/n
    S=0.0
    for i=1:(n-1)  #calcula o "meio"
        x=a+i*h
        S+=2*f(x)
    end
    S=h/2*(S+f(a)+f(b)) #calcula "as pontas"
    return S
end


trapezio (generic function with 1 method)

In [ ]:
# Exemplo 1
function circulo(x) 
    return 4*sqrt(1 - x^2)
end
resultado = trapezio(circulo, 1000000000, 0, 1)
abs(MathConstants.pi - resultado) < 10^(-12)

In [10]:
# Exemplo 2
#area do triangulo (0,3);(0,0);(4,0)
function triangulo(x) 
    return (-3/4)*x + 3
end
resultado = trapezio(triangulo, 10^(7), 0, 4)
abs((4*3)/2 - resultado) < 10^(-13)

true

In [21]:
# Exemplo 3
#integral de 3*x^2
function fa(x) 
    return 3*x^2
end

resultado = trapezio(fa, 10^(9), 2, 15)
integralMatematica = ((15^3) - (2^3))
abs((integralMatematica - resultado)) < 10^(-10)

true

## 19) Integração numérica com erro

In [23]:
#Entrada: uma função f, tolerancia de erro entre o valor real e a aproximação,  um intervalo [a,b], e por fim o teto da segunda derivada no intervalo dado
#A função irá utilizar o método do trapézio para calcular a aproximação integral da função enviada por parametro
#no intervalo [a, b], com a condição de que a diferença entre o valor real e a aproximação seja menor que a tolerância de erro
#Saida: aproximação integral de f no intervalo a[a, b]

function trapezioComErro(f, tol, a,b, M) #calcular a integral f(x) de a até b
    n=ceil(sqrt((M*(b-a)^3)/(12*tol)))
    
    h=(b-a)/n
    S=0.0
    for i=1:(n-1)  #calcula o "meio"
        x=a+i*h
        S+=2*f(x)
    end
    S=h/2*(S+f(a)+f(b)) #calcula "as pontas"
    return S
end


trapezioComErro (generic function with 1 method)

In [34]:
# Exemplo 1
#area do triangulo (0,3);(0,0);(4,0)
function triangulo(x) 
    return (-3/4)*x + 3
end
resultado = trapezioComErro(triangulo, 10^(-13), 0, 4, 0.1)
abs((4*3)/2 - resultado) < 10^(-13)

true

In [31]:
# Exemplo 2
#area do triangulo (0,3);(0,0);(4,0)
function fa(x) 
    return 3*x^2
end

resultado = trapezioComErro(fa, 10^(-9), 2, 15, 6)
integralMatematica = ((15^3) - (2^3))
abs((integralMatematica - resultado)) <= 10^(-9)

true

In [33]:
# Exemplo 3
#integral da função exponencial 
function exp1(x) 
    return MathConstants.e^x
end

resultado = trapezioComErro(exp1, 10^(-10), 2, 5, 149)
integralMatematica = (exp1(5) - exp1(2))
abs((integralMatematica - resultado)) <= 10^(-10)

true

## 20) Integração dupla

In [ ]:
# Entradas: f(x, y), g(y), h(y), b, a
# Saída: Integral dupla de f(x,y) dxdy
# A função calcula a integral de h(y) a g(y) de f(x,y) e a integral de a até b do resultado anterior, 
# por fim, nos retornando a integral dupla da função f recebida
function trapezio(f,a,b,n) #calcular a integral f(x) de a até b utilizando o método de trapezio
    h=(b-a)/n
    S=0.0
    for i=1:(n-1)  #calcula o "meio"
        x=a+i*h
        S+=2*f(x)
    end
    S=h/2*(S+f(a)+f(b)) #calcula "as pontas"
    return S
end

function Integral(f,a,b)
    return trapezio(f,a,b,1000)
end

function Integral_Dupla(h,a,b,c,d) #integral dupla de h(x,y) de a até b e de h(y) até g(y)
    function g(y)
        f(x) = h(x,y)
        return Integral(f,c(y),d(y))
    end
    return Integral(g,a,b)
end

In [ ]:
# Exemplo 1
h(x,y)=x^2*y
z(y) = 2*y
w(y) = 3*y
a = 5
b = 6
Integral_Dupla(h,a,b,z,w)


In [ ]:

# Exemplo 2
h(x,y)=2-y
z(y) = y^2
w(y) = sqrt(y)
a = 0
b = 1
Integral_Dupla(h,a,b,z,w)


In [ ]:

# Exemplo 3
h(x,y)=(y+x)^(-2)
z(y) = 0
w(y) = 1
a = 1
b = 2
Integral_Dupla(h,a,b,z,w)

## 21) Derivada numérica com função contínua

### i)

In [ ]:
function derivar_frente1(x, funcao, h)#funcao que calcula a derivada para frente
    function z(x)
        return (funcao(x+h)-funcao(x))/h

    end    
    return z
end    
    
function r(x)
    return x^3
end

d_f = derivar_frente1(2, r, 0.00001)
d_f(7)


### ii)

In [ ]:
function derivar_tras1(x, funcao, h)#funcao que calcula a derivada para trás    
    function z(x)
        return (funcao(x)-funcao(x-h))/h
    end    
    
    return z
end  

function r(x)
    return x^2
end

d_t = derivar_tras1(2, r, 0.00001)
print(exp(7))
d_t(7)


### iii)

In [ ]:
function derivar_centralizada1(x, funcao, h)#funcao que calcula a derivada utilizando as duas funções derivadas, somando as duas e dividindo por dois, para garantir um resultado mais acertivo
    function z(x)
        return (funcao(x+h)-funcao(x-h))/(2*h)
    end
    return z
end

function r(x)
    return x^2
end

d_c = derivar_centralizada1(2, r, 0.0000001)

d_c(7)

## 22) Derivada numérica com função discreta

## i)

In [ ]:
function derivar_frente2(f, x)#funcao que calcula a derivada para trás    
    n,= size(x)
    y=zeros(n-1,1)
    for i in 1:n-1
        y[i]=f[i+1]-f[i]
    end    
    return y
end  

f_df_2=[30000 32000 34000 36000 38000 40000 42000 44000 46000 48000 50000]
x=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
derivar_frente2(f_df_2, x)

## ii)

In [ ]:
function derivar_tras2(f, x)#funcao que calcula a derivada para trás    
    n,= size(x)
    y=zeros(n-1,1)
    for i in reverse(2:n)
        y[i-1]=f[i]-f[i-1]
    end    
    return y
end  

f_dt_2=[30000 32000 34000 36000 38000 40000 42000 44000 46000 48000 50000]
x=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
derivar_tras2(f_dt_2, x)

### iii)

In [ ]:
function derivar_centralizada2(f, x)#funcao que calcula a derivada utilizando as duas funções derivadas, somando as duas e dividindo por dois, para garantir um resultado mais acertivo
    n,= size(x)
    y=zeros(n-2,1)
    for i in 2:n-1
        y[i-1]=(f[i+1]-f[i-1])/2
    end    
    return y
end

f_dc_2=[30000 32000 34000 36000 38000 40000 42000 44000 46000 48000 50000]
x=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
derivar_centralizada2(f_dc_2, x)